###  importing relevant libraries

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix


### Discovering Labels

In [7]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    df["Label"]=df["Label"].replace(renamer)
    df=df.dropna()
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [8]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("Tthe folder could not be created!")

In [9]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [10]:
renamer={'Amazon Alexa Echo Dot 1': "Amazon Alexa Echo Dot",
 'Amazon Alexa Echo Dot 2': "Amazon Alexa Echo Dot",
 'D-Link DCHS-161 Water Sensor': 'D-Link Water Sensor',
 'DCS8000LHA1 D-Link Mini Camera': 'D-Link Mini Camera',
 'Gosund ESP_032979 Plug':'Gosund Plug',
 'Gosund ESP_039AAF Socket':'Gosund Socket',
 'Gosund ESP_0C3994 Plug': 'Gosund Plug',
 'Gosund ESP_10098F Socket':'Gosund Socket',
 'Gosund ESP_10ACD8 Plug':'Gosund Plug',
 'Gosund ESP_147FF9 Plug': 'Gosund Plug',
 'Gosund ESP_1ACEE1  Socket':'Gosund Socket',
 'Philips Hue Bridge 1': "Philips Hue Bridge",
 'Philips Hue Bridge 2': "Philips Hue Bridge",
 'Philips Hue White 1': "Philips Hue White",
 'Philips Hue White 2':"Philips Hue White",
 'Ring Base Station AC:1236': 'Ring Base Station',
 'Sengled Smart Plug 1': 'Sengled Smart Plug',
 'Sengled Smart Plug 2': 'Sengled Smart Plug',
 'SmartThings Smart Bulb 1': 'SmartThings Smart Bulb',
 'SmartThings Smart Bulb 2': 'SmartThings Smart Bulb',
 'SmartThings Smart Bulb 3': 'SmartThings Smart Bulb',
 'SmartThings Smart Bulb 4': 'SmartThings Smart Bulb',
 'SmartThings Smart Bulb 5': 'SmartThings Smart Bulb',
 'Sonoff Smart Plug 1': 'Sonoff Smart Plug',
 'Sonoff Smart Plug 2': 'Sonoff Smart Plug',
 'Teckin Plug 1':'Teckin Plug',
 'Teckin Plug 2':'Teckin Plug',
 'Yutron Plug 1': 'Yutron Plug',
 'Yutron Plug 2':  'Yutron Plug'}

In [11]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['MAC']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [12]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [13]:


def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]


In [14]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        repetition=25
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes


            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            graph_name=output_csv+"_confusion matrix.pdf"   
            plt.figure(figsize = (40,28))
            sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



# Machine learning applications 

# Aalto Dataset

In [11]:
feature= ['pck_size', 'Ether_type', 'LLC_ctrl', 'EAPOL_version', 'EAPOL_type', 'IP_ihl', 'IP_tos', 'IP_len', 'IP_flags', 'IP_DF', 'IP_ttl', 'IP_options', 'ICMP_code', 'TCP_dataofs', 'TCP_FIN', 'TCP_ACK', 'TCP_window', 'UDP_len', 'DHCP_options', 'BOOTP_hlen', 'BOOTP_flags', 'BOOTP_sname', 'BOOTP_file', 'BOOTP_options', 'DNS_qr', 'DNS_rd', 'DNS_qdcount', 'dport_class', 'payload_bytes', 'entropy',
"MAC",
'Label']


In [16]:
ml_list={"DT" :DecisionTreeClassifier()}


In [17]:
find_the_way("./","clean")

[]

In [18]:
file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD11T":['./small/DI-S1.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD12":['./small/AD-S1.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD12T":['./small/DI-S2.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD22":['./small/AD-S2.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD22T":['./small/DI-S2.csv','./small/AD-S2.csv','./small/AD-S1.csv'],
"DD21":['./small/AD-S2.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD21T":['./small/DI-S1.csv','./small/AD-S2.csv','./small/AD-S1.csv']}
# In[48]:

In [15]:
fdict={2: ['dstport',
  'dstport',
  'dstport_class',
  'http.chat',
  'http.notification',
  'http.request.method',
  'ip.flags',
  'ip.flags.df',
  'ip.len',
  'ip.opt.type',
  'ip.proto',
  'ip.ttl',
  'Payload_Entropy',
  'srcport',
  'srcport_class',
  'tcp.ack',
  'tcp.analysis.ack_rtt',
  'tcp.analysis.bytes_in_flight',
  'tcp.analysis.initial_rtt',
  'tcp.analysis.push_bytes_sent',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.push',
  'tcp.flags.str',
  'tcp.flags.syn',
  'tcp.hdr_len',
  'tcp.len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.time_delta',
  'tcp.time_relative',
  'tcp.window_size',
  'tcp.window_size_scalefactor',
  'tcp.window_size_value',
  'tls.record.length',
  'udp.checksum.status',
  'udp.dstport',
  'udp.length',
  'udp.srcport',
  'udp.time_delta',
  'udp.time_relative',"Label"],
 3: ['dstport',
  'dstport',
  'dstport_class',
  'http.chat',
  'http.notification',
  'http.request.method',
  'ip.flags.df',
  'ip.len',
  'ip.proto',
  'ip.ttl',
  'srcport',
  'srcport_class',
  'tcp.ack',
  'tcp.analysis.ack_rtt',
  'tcp.analysis.bytes_in_flight',
  'tcp.analysis.initial_rtt',
  'tcp.analysis.push_bytes_sent',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.push',
  'tcp.flags.str',
  'tcp.flags.syn',
  'tcp.hdr_len',
  'tcp.len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.time_delta',
  'tcp.time_relative',
  'tcp.window_size',
  'tcp.window_size_scalefactor',
  'tcp.window_size_value',
  'tls.record.length',
  'udp.checksum.status',
  'udp.dstport',
  'udp.srcport',
  'udp.time_delta',
  'udp.time_relative',"Label"],
 4: ['dstport_class',
  'http.chat',
  'http.notification',
  'http.request.method',
  'ip.flags.df',
  'ip.len',
  'ip.proto',
  'srcport_class',
  'tcp.ack',
  'tcp.analysis.bytes_in_flight',
  'tcp.analysis.push_bytes_sent',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.push',
  'tcp.flags.str',
  'tcp.flags.syn',
  'tcp.hdr_len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.time_delta',
  'tcp.window_size',
  'tcp.window_size_scalefactor',
  'tcp.window_size_value',
  'udp.srcport',
  'udp.time_delta',
  'udp.time_relative',"Label"],
 5: ['dstport_class',
  'http.chat',
  'http.request.method',
  'ip.proto',
  'srcport_class',
  'tcp.ack',
  'tcp.analysis.bytes_in_flight',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.push',
  'tcp.flags.str',
  'tcp.hdr_len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.time_delta',
  'tcp.window_size',
  'udp.srcport',
  'udp.time_relative',"Label"],
 6: ['http.chat',
  'http.request.method',
  'tcp.ack',
  'tcp.analysis.bytes_in_flight',
  'tcp.completeness',
  'tcp.flags',
  'tcp.flags.str',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.window_size',"Label"],
 7: ['tcp.nxtseq', 'tcp.window_size',"Label"]}

In [16]:
say=0
dataset="./results/other/1/"
folder(dataset)
for f in fdict:
    for ii in file_list:
            say+=1
            train=file_list[ii][0]
            test=file_list[ii][1]
            mixed=False
            step=1
            sayac=1
            output_csv=f"{dataset}/@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}_{f}.csv"
            print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}-{f}")
            target_names=target_name(test)
            ML(train,test,output_csv,fdict[f],step,mixed,dataset[2:-1]+"_"+str(f))   


AD-S1-DI-S1-2
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_2    0     1  DT    0.701    0.72    0.737  0.72   0.693  0.686      0    1.209     0.031          0
 1  results/other/1_2    1     2  DT    0.759    0.777   0.779  0.777  0.743  0.747      0    1.211     0.031          0
 2  results/other/1_2    2     3  DT    0.753    0.77    0.784  0.77   0.744  0.74       0    1.197     0.027          0
 3  results/other/1_2    3     4  DT    0.708    0.736   0.743  0.736  0.7    0.693      0    1.211     0.021          0
 4  results/other/1_2    4     5  DT    0.754    0.771   0.773  0.771  0.736  0.741      0    1.259     0.016          0
 5  results/other/1_2    5     6  DT    0.755    0.772   0.783  0.772  0.74   0.742      0    1.257     0.016          0
 6  results/other/

DI-S1-AD-S1-2
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_2    0     1  DT    0.655    0.661   0.688  0.661  0.608  0.636      0    1.244     0.023          0
 1  results/other/1_2    1     2  DT    0.654    0.66    0.685  0.66   0.612  0.635      0    1.174     0.032          0
 2  results/other/1_2    2     3  DT    0.655    0.662   0.683  0.662  0.61   0.636      0    1.192     0.016          0
 3  results/other/1_2    3     4  DT    0.659    0.669   0.725  0.669  0.647  0.64       0    1.052     0.032          0
 4  results/other/1_2    4     5  DT    0.655    0.663   0.683  0.663  0.612  0.636      0    1.14      0.028          0
 5  results/other/1_2    5     6  DT    0.643    0.652   0.688  0.652  0.606  0.623      0    1.151     0.031          0
 6  results/other/

AD-S1-DI-S2-2
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_2    0     1  DT    0.745    0.746   0.764  0.746  0.721  0.731      0    1.097     0.03           0
 1  results/other/1_2    1     2  DT    0.717    0.727   0.755  0.727  0.696  0.702      0    1.078     0.032          0
 2  results/other/1_2    2     3  DT    0.726    0.734   0.758  0.734  0.71   0.711      0    1.151     0.031          0
 3  results/other/1_2    3     4  DT    0.726    0.733   0.751  0.733  0.704  0.711      0    1.143     0.031          0
 4  results/other/1_2    4     5  DT    0.721    0.729   0.752  0.729  0.701  0.705      0    1.116     0.036          0
 5  results/other/1_2    5     6  DT    0.733    0.74    0.754  0.74   0.713  0.719      0    1.115     0.024          0
 6  results/other/

DI-S2-AD-S1-2
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_2    0     1  DT    0.671    0.691   0.702  0.691  0.651  0.653      0    1.416     0.016          0
 1  results/other/1_2    1     2  DT    0.665    0.684   0.693  0.684  0.65   0.646      0    1.349     0.031          0
 2  results/other/1_2    2     3  DT    0.66     0.68    0.681  0.68   0.639  0.642      0    1.442     0.031          0
 3  results/other/1_2    3     4  DT    0.703    0.726   0.732  0.726  0.689  0.687      0    1.38      0.031          0
 4  results/other/1_2    4     5  DT    0.701    0.725   0.759  0.725  0.714  0.684      0    1.338     0.016          0
 5  results/other/1_2    5     6  DT    0.701    0.725   0.738  0.725  0.692  0.685      0    1.253     0.031          0
 6  results/other/

AD-S2-DI-S2-2
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_2    0     1  DT    0.701    0.697   0.683  0.697  0.648  0.684      0    1.019     0.016          0
 1  results/other/1_2    1     2  DT    0.695    0.692   0.664  0.692  0.64   0.679      0    1.847     0.034          0
 2  results/other/1_2    2     3  DT    0.685    0.682   0.659  0.682  0.631  0.668      0    1.012     0.031          0
 3  results/other/1_2    3     4  DT    0.701    0.697   0.672  0.697  0.647  0.685      0    0.983     0.016          0
 4  results/other/1_2    4     5  DT    0.701    0.697   0.664  0.697  0.645  0.685      0    1.005     0.031          0
 5  results/other/1_2    5     6  DT    0.699    0.695   0.67   0.695  0.643  0.683      0    0.942     0.016          0
 6  results/other/

...

DI-S2-AD-S1-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_7    0     1  DT    0.47     0.511   0.706  0.511  0.512  0.446      0    0.133     0.015          0
 1  results/other/1_7    1     2  DT    0.471    0.512   0.708  0.512  0.513  0.447      0    0.123     0.016          0
 2  results/other/1_7    2     3  DT    0.476    0.518   0.745  0.518  0.527  0.453      0    0.129     0.016          0
 3  results/other/1_7    3     4  DT    0.472    0.513   0.699  0.513  0.514  0.448      0    0.119     0.016          0
 4  results/other/1_7    4     5  DT    0.472    0.513   0.699  0.513  0.514  0.448      0    0.121     0.016          0
 5  results/other/1_7    5     6  DT    0.472    0.512   0.706  0.512  0.514  0.448      0    0.121     0.017          0
 6  results/other/

AD-S2-DI-S2-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_7    0     1  DT    0.495    0.51    0.74   0.51   0.537  0.465      0    0.081     0.015          0
 1  results/other/1_7    1     2  DT    0.495    0.51    0.741  0.51   0.537  0.465      0    0.082     0.017          0
 2  results/other/1_7    2     3  DT    0.495    0.51    0.74   0.51   0.537  0.465      0    0.08      0.014          0
 3  results/other/1_7    3     4  DT    0.495    0.51    0.741  0.51   0.537  0.465      0    0.083     0.016          0
 4  results/other/1_7    4     5  DT    0.496    0.511   0.742  0.511  0.538  0.467      0    0.081     0.017          0
 5  results/other/1_7    5     6  DT    0.496    0.511   0.742  0.511  0.538  0.466      0    0.074     0.016          0
 6  results/other/

DI-S2-AD-S2-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_7    0     1  DT    0.478    0.515   0.778  0.515  0.548  0.456      0    0.128     0.015          0
 1  results/other/1_7    1     2  DT    0.474    0.511   0.725  0.511  0.534  0.451      0    0.126     0.015          0
 2  results/other/1_7    2     3  DT    0.479    0.516   0.778  0.516  0.547  0.456      0    0.111     0.013          0
 3  results/other/1_7    3     4  DT    0.48     0.518   0.779  0.518  0.551  0.458      0    0.115     0.015          0
 4  results/other/1_7    4     5  DT    0.48     0.517   0.779  0.517  0.55   0.457      0    0.124     0.014          0
 5  results/other/1_7    5     6  DT    0.479    0.516   0.778  0.516  0.548  0.456      0    0.117     0.015          0
 6  results/other/

AD-S2-DI-S1-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_7    0     1  DT    0.496    0.519   0.764  0.519  0.54   0.467      0    0.077     0.015          0
 1  results/other/1_7    1     2  DT    0.495    0.518   0.762  0.518  0.539  0.466      0    0.076     0.016          0
 2  results/other/1_7    2     3  DT    0.494    0.517   0.764  0.517  0.538  0.465      0    0.087     0.016          0
 3  results/other/1_7    3     4  DT    0.495    0.518   0.764  0.518  0.539  0.466      0    0.075     0.015          0
 4  results/other/1_7    4     5  DT    0.495    0.518   0.762  0.518  0.539  0.466      0    0.074     0.015          0
 5  results/other/1_7    5     6  DT    0.496    0.519   0.766  0.519  0.54   0.467      0    0.08      0.015          0
 6  results/other/

DI-S1-AD-S2-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_7    0     1  DT    0.481    0.512   0.721  0.512  0.521  0.457      0    0.106     0.015          0
 1  results/other/1_7    1     2  DT    0.481    0.512   0.719  0.512  0.521  0.457      0    0.109     0.014          0
 2  results/other/1_7    2     3  DT    0.479    0.509   0.719  0.509  0.517  0.455      0    0.111     0.014          0
 3  results/other/1_7    3     4  DT    0.479    0.509   0.72   0.509  0.517  0.455      0    0.113     0.015          0
 4  results/other/1_7    4     5  DT    0.478    0.509   0.718  0.509  0.517  0.454      0    0.109     0.015          0
 5  results/other/1_7    5     6  DT    0.479    0.509   0.718  0.509  0.517  0.455      0    0.113     0.014          0
 6  results/other/

In [19]:
features=[['dstport_class',
  'http.chat',
  'http.notification',
  'http.request.method',
  'ip.flags.df',
  'ip.proto',
  'srcport',
  'srcport_class',
  'tcp.ack',
  'tcp.analysis.ack_rtt',
  'tcp.analysis.bytes_in_flight',
  'tcp.flags',
  'tcp.flags.push',
  'tcp.flags.str',
  'tcp.flags.syn',
  'tcp.len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.time_delta',
  'tcp.window_size',
  'tcp.window_size_scalefactor',
  'udp.checksum.status',
  'udp.srcport',
  'udp.time_relative',"Label"],
 ['dstport',
  'dstport_class',
  'http.chat',
  'http.request.method',
  'ip.proto',
  'tcp.ack',
  'tcp.analysis.push_bytes_sent',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.push',
  'tcp.hdr_len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.window_size',
  'tcp.window_size_scalefactor',
  'udp.checksum.status',
  'udp.srcport',
  'udp.time_relative',"Label"],
 ['dstport_class',
  'http.chat',
  'http.notification',
  'http.request.method',
  'ip.flags.df',
  'ip.len',
  'tcp.ack',
  'tcp.analysis.ack_rtt',
  'tcp.analysis.bytes_in_flight',
  'tcp.analysis.initial_rtt',
  'tcp.analysis.push_bytes_sent',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.str',
  'tcp.flags.syn',
  'tcp.hdr_len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.time_delta',
  'tcp.window_size',
  'udp.checksum.status',
  'udp.dstport',"Label"],
 ['http.chat',
  'http.notification',
  'ip.flags',
  'ip.len',
  'ip.opt.type',
  'ip.proto',
  'srcport_class',
  'tcp.analysis.initial_rtt',
  'tcp.completeness',
  'tcp.flags.ack',
  'tcp.flags',
  'tcp.flags.str',
  'tcp.len',
  'tcp.stream',
  'tcp.time_relative',
  'tcp.window_size',
  'tcp.window_size_value',"Label"],
 ['dstport_class',
  'ip.len',
  'ip.opt.type',
  'ip.ttl',
  'srcport',
  'srcport_class',
  'tcp.ack',
  'tcp.analysis.bytes_in_flight',
  'tcp.analysis.initial_rtt',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.str',
  'tcp.hdr_len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.time_delta',
  'tcp.window_size',
  'tcp.window_size_scalefactor',
  'tcp.window_size_value',
  'tls.record.length',
  'udp.length',
  'udp.srcport',
  'udp.time_delta',
  'udp.time_relative',"Label"],
 ['dstport_class',
  'http.chat',
  'http.notification',
  'http.request.method',
  'ip.ttl',
  'Payload_Entropy',
  'srcport',
  'tcp.ack',
  'tcp.analysis.bytes_in_flight',
  'tcp.analysis.push_bytes_sent',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags.push',
  'tcp.flags.str',
  'tcp.flags.syn',
  'tcp.hdr_len',
  'tcp.len',
  'tcp.nxtseq',
  'tcp.options',
  'tcp.stream',
  'tcp.time_relative',
  'tcp.window_size',
  'tcp.window_size_scalefactor',
  'tls.record.length',
  'udp.dstport',
  'udp.time_delta',
  'udp.time_relative',"Label"],
 ['dstport',
  'http.chat',
  'http.request.method',
  'ip.flags.df',
  'ip.proto',
  'ip.ttl',
  'Payload_Entropy',
  'srcport_class',
  'tcp.analysis.ack_rtt',
  'tcp.analysis.bytes_in_flight',
  'tcp.analysis.push_bytes_sent',
  'tcp.completeness',
  'tcp.flags.push',
  'tcp.flags.syn',
  'tcp.nxtseq',
  'tcp.seq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.time_delta',
  'tcp.window_size_value',
  'udp.dstport',
  'udp.srcport',
  'udp.time_delta',
  'udp.time_relative',"Label"],
 ['dstport',
  'http.request.method',
  'ip.flags',
  'ip.flags.df',
  'ip.len',
  'ip.proto',
  'srcport_class',
  'tcp.ack',
  'tcp.analysis.bytes_in_flight',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.push',
  'tcp.flags.str',
  'tcp.hdr_len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.time_delta',
  'tcp.time_relative',
  'tcp.window_size',
  'tcp.window_size_value',
  'tls.record.length',
  'udp.length',
  'udp.srcport',
  'udp.time_delta',"Label"]]

In [3]:
len(features)

8

In [19]:
say=0
dataset="./results/other/2/"
folder(dataset)
for ff,f in enumerate( features):
    for ii in file_list:
            say+=1
            train=file_list[ii][0]
            test=file_list[ii][1]
            mixed=False
            step=1
            sayac=1
            output_csv=f"{dataset}/@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}_{ff}.csv"
            print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}-{ff}")
            target_names=target_name(test)
            ML(train,test,output_csv,f,step,mixed,dataset[2:-1]+"_"+str(ff))   


AD-S1-DI-S1-0
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_0    0     1  DT    0.672    0.674   0.743  0.674  0.661  0.655      0    0.728     0.023          0
 1  results/other/2_0    1     2  DT    0.676    0.697   0.756  0.697  0.681  0.659      0    0.686     0.024          0
 2  results/other/2_0    2     3  DT    0.668    0.671   0.731  0.671  0.655  0.65       0    0.701     0.024          0
 3  results/other/2_0    3     4  DT    0.67     0.692   0.743  0.692  0.674  0.652      0    0.685     0.022          0
 4  results/other/2_0    4     5  DT    0.672    0.676   0.733  0.676  0.659  0.655      0    0.704     0.023          0
 5  results/other/2_0    5     6  DT    0.668    0.689   0.739  0.689  0.67   0.649      0    0.72      0.027          0
 6  results/other/

DI-S1-AD-S1-0
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_0    0     1  DT    0.607    0.605   0.624  0.605  0.558  0.585      0    0.693     0.024          0
 1  results/other/2_0    1     2  DT    0.605    0.604   0.643  0.604  0.556  0.583      0    0.713     0.023          0
 2  results/other/2_0    2     3  DT    0.627    0.625   0.637  0.625  0.574  0.606      0    0.675     0.025          0
 3  results/other/2_0    3     4  DT    0.618    0.615   0.632  0.615  0.568  0.597      0    0.685     0.023          0
 4  results/other/2_0    4     5  DT    0.617    0.614   0.647  0.614  0.564  0.596      0    0.68      0.023          0
 5  results/other/2_0    5     6  DT    0.608    0.607   0.685  0.607  0.585  0.585      0    0.677     0.022          0
 6  results/other/

AD-S1-DI-S2-0
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_0    0     1  DT    0.651    0.648   0.672  0.648  0.605  0.632      0    0.718     0.023          0
 1  results/other/2_0    1     2  DT    0.653    0.648   0.677  0.648  0.606  0.634      0    0.707     0.022          0
 2  results/other/2_0    2     3  DT    0.653    0.649   0.667  0.649  0.603  0.634      0    0.716     0.023          0
 3  results/other/2_0    3     4  DT    0.655    0.651   0.673  0.651  0.607  0.636      0    0.707     0.023          0
 4  results/other/2_0    4     5  DT    0.648    0.644   0.673  0.644  0.604  0.629      0    0.694     0.024          0
 5  results/other/2_0    5     6  DT    0.648    0.645   0.671  0.645  0.603  0.629      0    0.719     0.023          0
 6  results/other/

DI-S2-AD-S1-0
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_0    0     1  DT    0.653    0.669   0.668  0.669  0.627  0.635      0    0.743     0.023          0
 1  results/other/2_0    1     2  DT    0.649    0.664   0.668  0.664  0.624  0.63       0    0.738     0.023          0
 2  results/other/2_0    2     3  DT    0.657    0.672   0.685  0.672  0.637  0.638      0    0.75      0.023          0
 3  results/other/2_0    3     4  DT    0.65     0.665   0.671  0.665  0.626  0.631      0    0.762     0.023          0
 4  results/other/2_0    4     5  DT    0.653    0.67    0.683  0.67   0.633  0.634      0    0.607     0.024          0
 5  results/other/2_0    5     6  DT    0.659    0.675   0.687  0.675  0.639  0.641      0    0.753     0.024          0
 6  results/other/

AD-S2-DI-S2-0
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_0    0     1  DT    0.647    0.656   0.669  0.656  0.606  0.627      0    0.637     0.022          0
 1  results/other/2_0    1     2  DT    0.649    0.658   0.666  0.658  0.607  0.63       0    0.64      0.023          0
 2  results/other/2_0    2     3  DT    0.632    0.644   0.635  0.644  0.583  0.612      0    0.672     0.027          0
 3  results/other/2_0    3     4  DT    0.664    0.673   0.669  0.673  0.623  0.646      0    0.478     0.02           0
 4  results/other/2_0    4     5  DT    0.643    0.655   0.642  0.655  0.593  0.623      0    0.636     0.022          0
 5  results/other/2_0    5     6  DT    0.646    0.655   0.672  0.655  0.605  0.627      0    0.629     0.022          0
 6  results/other/

...

DI-S2-AD-S2-6
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_6    0     1  DT    0.691    0.704   0.731  0.704  0.692  0.674      0    0.816     0.016          0
 1  results/other/2_6    1     2  DT    0.676    0.694   0.688  0.694  0.66   0.659      0    0.831     0.016          0
 2  results/other/2_6    2     3  DT    0.701    0.717   0.716  0.717  0.684  0.685      0    0.81      0.018          0
 3  results/other/2_6    3     4  DT    0.679    0.694   0.695  0.694  0.664  0.662      0    0.817     0.02           0
 4  results/other/2_6    4     5  DT    0.68     0.698   0.716  0.698  0.689  0.663      0    0.771     0.019          0
 5  results/other/2_6    5     6  DT    0.696    0.712   0.734  0.712  0.698  0.68       0    0.805     0.016          0
 6  results/other/

AD-S2-DI-S1-6
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_6    0     1  DT    0.69     0.703   0.763  0.703  0.678  0.673      0    0.677     0.016          0
 1  results/other/2_6    1     2  DT    0.682    0.694   0.766  0.694  0.668  0.665      0    0.693     0.031          0
 2  results/other/2_6    2     3  DT    0.689    0.702   0.762  0.702  0.678  0.671      0    0.721     0.016          0
 3  results/other/2_6    3     4  DT    0.685    0.697   0.769  0.697  0.672  0.668      0    0.691     0.031          0
 4  results/other/2_6    4     5  DT    0.686    0.697   0.753  0.697  0.671  0.669      0    0.693     0.047          0
 5  results/other/2_6    5     6  DT    0.685    0.697   0.769  0.697  0.674  0.668      0    1.394     0.047          0
 6  results/other/

DI-S1-AD-S2-6
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_6    0     1  DT    0.65     0.648   0.683  0.648  0.613  0.631      0    0.758     0.016          0
 1  results/other/2_6    1     2  DT    0.65     0.646   0.683  0.646  0.61   0.631      0    0.747     0.009          0
 2  results/other/2_6    2     3  DT    0.651    0.648   0.706  0.648  0.642  0.631      0    0.748     0.016          0
 3  results/other/2_6    3     4  DT    0.637    0.638   0.668  0.638  0.604  0.618      0    0.724     0.016          0
 4  results/other/2_6    4     5  DT    0.645    0.637   0.665  0.637  0.607  0.626      0    0.759     0.012          0
 5  results/other/2_6    5     6  DT    0.647    0.645   0.7    0.645  0.639  0.628      0    0.742     0.028          0
 6  results/other/

AD-S1-DI-S1-7


KeyError: 'Label'

In [20]:
say=0
dataset="./results/other/2/"
folder(dataset)
for ff in range(7,len(features)):
    for ii in file_list:
            say+=1
            train=file_list[ii][0]
            test=file_list[ii][1]
            mixed=False
            step=1
            sayac=1
            output_csv=f"{dataset}/@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}_{ff}.csv"
            print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}-{ff}")
            target_names=target_name(test)
            ML(train,test,output_csv,features[ff],step,mixed,dataset[2:-1]+"_"+str(ff))   


AD-S1-DI-S1-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_7    0     1  DT    0.677    0.706   0.745  0.706  0.676  0.66       0    0.745     0.018          0
 1  results/other/2_7    1     2  DT    0.681    0.709   0.739  0.709  0.673  0.664      0    0.759     0.019          0
 2  results/other/2_7    2     3  DT    0.678    0.707   0.744  0.707  0.677  0.662      0    0.722     0.017          0
 3  results/other/2_7    3     4  DT    0.678    0.706   0.745  0.706  0.676  0.661      0    0.737     0.018          0
 4  results/other/2_7    4     5  DT    0.679    0.707   0.748  0.707  0.677  0.662      0    0.694     0.023          0
 5  results/other/2_7    5     6  DT    0.678    0.707   0.746  0.707  0.676  0.661      0    0.735     0.031          0
 6  results/other/

DI-S1-AD-S1-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_7    0     1  DT    0.644    0.651   0.651  0.651  0.606  0.625      0    0.802     0.016          0
 1  results/other/2_7    1     2  DT    0.647    0.653   0.649  0.653  0.608  0.627      0    0.831     0.031          0
 2  results/other/2_7    2     3  DT    0.647    0.653   0.641  0.653  0.612  0.627      0    0.834     0.018          0
 3  results/other/2_7    3     4  DT    0.649    0.655   0.654  0.655  0.611  0.63       0    0.811     0.016          0
 4  results/other/2_7    4     5  DT    0.649    0.654   0.654  0.654  0.61   0.63       0    0.787     0.035          0
 5  results/other/2_7    5     6  DT    0.646    0.652   0.642  0.652  0.608  0.626      0    0.812     0.016          0
 6  results/other/

AD-S1-DI-S2-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_7    0     1  DT    0.625    0.645   0.668  0.645  0.605  0.605      0    0.758     0.031          0
 1  results/other/2_7    1     2  DT    0.623    0.645   0.657  0.645  0.605  0.604      0    0.788     0.016          0
 2  results/other/2_7    2     3  DT    0.656    0.677   0.711  0.677  0.643  0.638      0    0.764     0.016          0
 3  results/other/2_7    3     4  DT    0.661    0.682   0.712  0.682  0.644  0.643      0    0.765     0.016          0
 4  results/other/2_7    4     5  DT    0.658    0.678   0.699  0.678  0.64   0.64       0    0.771     0.031          0
 5  results/other/2_7    5     6  DT    0.63     0.652   0.668  0.652  0.61   0.611      0    0.757     0.031          0
 6  results/other/

DI-S2-AD-S1-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_7    0     1  DT    0.638    0.656   0.673  0.656  0.625  0.618      0    0.826     0.017          0
 1  results/other/2_7    1     2  DT    0.647    0.666   0.687  0.666  0.638  0.627      0    0.812     0.023          0
 2  results/other/2_7    2     3  DT    0.646    0.664   0.692  0.664  0.641  0.625      0    0.833     0.016          0
 3  results/other/2_7    3     4  DT    0.658    0.677   0.701  0.677  0.643  0.639      0    0.818     0.016          0
 4  results/other/2_7    4     5  DT    0.654    0.674   0.705  0.674  0.644  0.635      0    0.81      0.031          0
 5  results/other/2_7    5     6  DT    0.644    0.662   0.699  0.662  0.643  0.624      0    0.819     0.021          0
 6  results/other/

AD-S2-DI-S2-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_7    0     1  DT    0.633    0.63    0.64   0.63   0.592  0.613      0    0.677     0.031          0
 1  results/other/2_7    1     2  DT    0.646    0.654   0.66   0.654  0.608  0.627      0    0.675     0.016          0
 2  results/other/2_7    2     3  DT    0.644    0.652   0.651  0.652  0.604  0.625      0    0.653     0.016          0
 3  results/other/2_7    3     4  DT    0.645    0.653   0.637  0.653  0.598  0.626      0    0.661     0.031          0
 4  results/other/2_7    4     5  DT    0.636    0.646   0.646  0.646  0.596  0.617      0    0.675     0.023          0
 5  results/other/2_7    5     6  DT    0.64     0.635   0.642  0.635  0.602  0.62       0    0.68      0.031          0
 6  results/other/

DI-S2-AD-S2-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_7    0     1  DT    0.691    0.712   0.711  0.712  0.686  0.674      0    0.78      0.032          0
 1  results/other/2_7    1     2  DT    0.691    0.711   0.708  0.711  0.686  0.674      0    0.734     0.031          0
 2  results/other/2_7    2     3  DT    0.693    0.713   0.707  0.713  0.692  0.677      0    0.763     0.022          0
 3  results/other/2_7    3     4  DT    0.676    0.698   0.724  0.698  0.665  0.659      0    0.757     0.027          0
 4  results/other/2_7    4     5  DT    0.687    0.708   0.72   0.708  0.686  0.67       0    0.796     0.031          0
 5  results/other/2_7    5     6  DT    0.67     0.691   0.669  0.691  0.658  0.652      0    0.791     0.016          0
 6  results/other/

AD-S2-DI-S1-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_7    0     1  DT    0.638    0.649   0.634  0.649  0.59   0.619      0    0.608     0.018          0
 1  results/other/2_7    1     2  DT    0.646    0.659   0.653  0.659  0.596  0.628      0    0.645     0.016          0
 2  results/other/2_7    2     3  DT    0.635    0.65    0.633  0.65   0.592  0.616      0    0.631     0.016          0
 3  results/other/2_7    3     4  DT    0.642    0.656   0.637  0.656  0.596  0.624      0    0.648     0.016          0
 4  results/other/2_7    4     5  DT    0.643    0.656   0.651  0.656  0.596  0.624      0    0.523     0.021          0
 5  results/other/2_7    5     6  DT    0.657    0.668   0.642  0.668  0.605  0.639      0    0.649     0.016          0
 6  results/other/

DI-S1-AD-S2-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_7    0     1  DT    0.665    0.676   0.682  0.676  0.63   0.647      0    0.787     0.031          0
 1  results/other/2_7    1     2  DT    0.689    0.697   0.694  0.697  0.656  0.672      0    0.787     0.031          0
 2  results/other/2_7    2     3  DT    0.666    0.676   0.67   0.676  0.636  0.648      0    0.771     0.031          0
 3  results/other/2_7    3     4  DT    0.689    0.697   0.701  0.697  0.65   0.673      0    0.787     0.016          0
 4  results/other/2_7    4     5  DT    0.666    0.676   0.664  0.676  0.634  0.647      0    0.803     0.016          0
 5  results/other/2_7    5     6  DT    0.693    0.702   0.694  0.702  0.654  0.677      0    0.802     0.032          0
 6  results/other/

In [ ]:
df=pd.read_csv(file_list[ii][0],usecols=f)

In [ ]:
df.to_csv("ornek.csv")